In [ ]:
#rstanがインストールされていないならインストールする
x<-installed.packages()
if(any(x[,1]== 'rstan') == FALSE){
    system("add-apt-repository -y ppa:marutter/rrutter3.5")
    system("add-apt-repository -y ppa:marutter/c2d4u3.5") 
    system("apt-get update")
    system("apt install -y r-cran-rstan")
}

In [ ]:
library(rstan)
#表3.1のデータ、クラスA、クラスBの順に入力
mathA <- c(49,66,69,55,54,72,51,76,40,62,66,51,59,68,66,57,53,66,58,57)
mathB <- c(41,55,21,49,53,50,52,67,54,69,57,48,31,52,56,50,46,38,62,59)

In [ ]:
# ここでstanのコードをstancodeという変数に入れます
# 以下は 1group.stan の例
stancode <- '
data {
  int n1; // 第1群のデータ数
  int n2; // 第2群のデータ数
  real x1[n1]; // 第1群のデータ
  real x2[n2]; // 第2群のデータ
}

parameters {
  real mu1; // 第1群の平均μ1
  real mu2; // 第2群の平均μ2
  real<lower=0> sigma; // 第1群、第2群の標準偏差σ
}

model {
  mu1 ~ uniform(0,100);
  mu2 ~ uniform(0,100);
  sigma ~ uniform(0,50);

  x1 ~ normal(mu1,sigma);
  x2 ~ normal(mu2,sigma);
}

generated quantities {
  real mu1_minus_mu2;
  real u_mu1_gt_mu2;
  real u_mu1_minus_mu2_gt_5;
  real delta;
  real u_delta_gt_03;
  real U3;
  real u_U3_gt_06;
  real pi_d;
  real u_pi_d_gt_08;

  mu1_minus_mu2 = mu1 - mu2; // μ1-μ2 式(3.11)
  u_mu1_gt_mu2 = mu1 - mu2 > 0; // u_{μ1>μ2} 式(3.12)
  u_mu1_minus_mu2_gt_5 = mu1 - mu2 > 5; // u_{μ1-μ2>c} 式(3.13)
  delta = (mu1 - mu2) / sigma; // δ 式(3.15)
  u_delta_gt_03 = delta > 0.3; // u_{δ>c} 式(3.16)
  U3 = normal_cdf(mu1, mu2, sigma); // U3 式(3.17)
  u_U3_gt_06 = U3 > 0.6; // u_{U3>c} 式(3.21)
  pi_d = normal_cdf(delta / sqrt(2), 0, 1); // πd 式(3.24)
  u_pi_d_gt_08 = pi_d > 0.8; // u_{πd>c} 式(3.26)
}
'

In [ ]:
fit <- stan("2groups-indep.stan",
            data=list(n1=length(mathA),
                      n2=length(mathB),
                      x1=mathA,
                      x2=mathB),
            iter=21000,warmup=1000,chains=5)

In [ ]:
# パラメータの事後分布（推測結果）
print(fit,probs=c(0.025,0.05,0.95,0.975))

In [ ]:
# 発生させたパラメータのヒストグラム
stan_hist(fit,pars=c("mu1","mu2","sigma"))

In [ ]:
# パラメータの事後分布（小数以下3桁表示）
print(fit,probs=c(0.025,0.05,0.95,0.975),digits=3)